In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
190,AFG,Asia,Afghanistan,2020-07-18,35289.0,219.0,1147.0,32.0,906.512,5.626,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83
322,ALB,Europe,Albania,2020-07-18,3906.0,55.0,107.0,3.0,1357.287,19.112,...,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57
518,DZA,Africa,Algeria,2020-07-18,21948.0,593.0,1057.0,5.0,500.513,13.523,...,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-07-17,France,174766.0,0.0,30215.0,0.0,0.0,0.0
0,2020-07-17,China,85393.0,0.0,4653.0,0.0,0.0,0.0
0,2020-07-17,Italy,244028.0,0.0,35053.0,0.0,0.0,0.0
0,2020-07-17,Spain,259705.0,0.0,28431.0,0.0,0.0,0.0
0,2020-07-17,United States,3632638.0,0.0,139379.0,0.0,0.0,0.0
0,2020-07-17,World,14019472.0,0.0,595868.0,0.0,0.0,0.0
0,2020-07-17,United Kingdom,293472.0,0.0,45252.0,0.0,0.0,0.0
0,2020-07-17,Germany,201420.0,0.0,9097.0,0.0,0.0,0.0
0,2020-07-17,Iran,270411.0,0.0,13783.0,0.0,0.0,0.0
0,2020-07-17,Turkey,217785.0,0.0,5456.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-07-18,France,174766.0,174674.0,30215.0,30152.0,92.0,63.0
0,2020-07-18,China,85393.0,85403.0,4653.0,4645.0,-10.0,8.0
0,2020-07-18,Italy,244028.0,243967.0,35053.0,35028.0,61.0,25.0
0,2020-07-17,Spain,259705.0,260255.0,28431.0,28420.0,-550.0,11.0
0,2020-07-18,United States,3632638.0,3647715.0,139379.0,139266.0,-15077.0,113.0
0,2020-07-18,World,14019472.0,14044869.0,595868.0,597148.0,-25397.0,-1280.0
0,2020-07-18,United Kingdom,293472.0,293239.0,45252.0,45233.0,233.0,19.0
0,2020-07-18,Germany,201420.0,201372.0,9097.0,9083.0,48.0,14.0
0,2020-07-18,Iran,270411.0,269440.0,13783.0,13791.0,971.0,-8.0
0,2020-07-18,Turkey,217785.0,217799.0,5456.0,5458.0,-14.0,-2.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")